In [ ]:
%lsmagic

## Data Preparation

In [1]:
import os 
import sys

from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy
import pandas
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

WORKING_DIR_PATH = globals()['_dh'][0]
WORKING_DIR_PARENT_PATH = os.path.dirname(WORKING_DIR_PATH)
sys.path.insert(1, WORKING_DIR_PARENT_PATH)
from custom_module.utilities import *

In [2]:
tf.__version__

'2.2.0'

In [3]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features_1.csv', index_col=0)
features

,25th_percentile_db4_cA4,25th_percentile_db4_cD1,25th_percentile_db4_cD2,25th_percentile_db4_cD3,25th_percentile_db4_cD4,25th_percentile_db5_cA4,25th_percentile_db5_cD1,25th_percentile_db5_cD2,25th_percentile_db5_cD3,25th_percentile_db5_cD4,...,zcr_db5_cD3,zcr_db5_cD4,zcr_db8_cA7,zcr_db8_cD1,zcr_db8_cD2,zcr_db8_cD3,zcr_db8_cD4,zcr_db8_cD5,zcr_db8_cD6,zcr_db8_cD7
0,-0.236240,-0.012063,-0.027807,-0.051443,-0.082820,-0.237137,-0.011746,-0.027222,-0.051066,-0.082349,...,48729.0,24515.0,1367.0,236539.0,100477.0,48721.0,24523.0,10601.0,5868.0,2136.0
1,-0.733787,-0.017003,-0.045150,-0.053358,-0.045559,-0.728479,-0.016115,-0.044458,-0.053728,-0.045010,...,40467.0,21751.0,4280.0,232788.0,104560.0,40835.0,21922.0,11688.0,5678.0,4187.0
2,-0.206548,-0.001990,-0.004977,-0.010344,-0.032882,-0.206663,-0.001933,-0.004869,-0.009837,-0.031671,...,51110.0,28460.0,3429.0,238953.0,98627.0,50426.0,28197.0,12380.0,5750.0,2853.0
3,-0.505051,-0.017088,-0.050502,-0.085439,-0.155965,-0.503971,-0.016430,-0.050478,-0.080363,-0.153229,...,46009.0,24784.0,4084.0,239878.0,106315.0,46383.0,25001.0,10550.0,6405.0,3401.0
4,-0.265025,-0.007068,-0.017015,-0.033292,-0.048008,-0.263057,-0.006823,-0.016241,-0.032975,-0.046946,...,50126.0,21473.0,3355.0,235508.0,96446.0,50054.0,21542.0,11100.0,6028.0,3176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,-0.104572,-0.000267,-0.001287,-0.004655,-0.018606,-0.104197,-0.000241,-0.001230,-0.004309,-0.018012,...,111718.0,57071.0,5266.0,478728.0,194404.0,110085.0,56729.0,26120.0,10705.0,4714.0
3333,-0.064313,-0.000214,-0.000820,-0.004044,-0.013585,-0.064493,-0.000202,-0.000685,-0.003693,-0.013236,...,100899.0,57721.0,5961.0,378640.0,180357.0,98690.0,57296.0,25947.0,13375.0,3125.0
3334,-0.477425,-0.000967,-0.004003,-0.008481,-0.038837,-0.476932,-0.000873,-0.004009,-0.007479,-0.036333,...,95367.0,62597.0,3067.0,491052.0,210852.0,92628.0,61759.0,24119.0,12008.0,6321.0
3335,-0.328624,-0.001875,-0.008336,-0.021885,-0.049970,-0.329515,-0.001693,-0.008131,-0.021180,-0.048207,...,101933.0,50380.0,5290.0,483699.0,219884.0,100631.0,49950.0,30214.0,9810.0,5397.0


In [4]:
# Shuffle dataset
features = features.reindex(numpy.random.permutation(features.index))
features

,25th_percentile_db4_cA4,25th_percentile_db4_cD1,25th_percentile_db4_cD2,25th_percentile_db4_cD3,25th_percentile_db4_cD4,25th_percentile_db5_cA4,25th_percentile_db5_cD1,25th_percentile_db5_cD2,25th_percentile_db5_cD3,25th_percentile_db5_cD4,...,zcr_db5_cD3,zcr_db5_cD4,zcr_db8_cA7,zcr_db8_cD1,zcr_db8_cD2,zcr_db8_cD3,zcr_db8_cD4,zcr_db8_cD5,zcr_db8_cD6,zcr_db8_cD7
2062,-0.465859,-0.010896,-0.059665,-0.205795,-0.355809,-0.465991,-0.009601,-0.056382,-0.206757,-0.348595,...,111280.0,42379.0,6619.0,462474.0,229780.0,111213.0,42349.0,20872.0,10708.0,5930.0
2012,-0.753223,-0.007865,-0.028152,-0.076519,-0.153791,-0.751413,-0.007456,-0.028054,-0.074464,-0.148392,...,103911.0,48949.0,4716.0,496409.0,205594.0,102934.0,49009.0,23687.0,10963.0,5791.0
1154,-0.563173,-0.006111,-0.022889,-0.092847,-0.198442,-0.560591,-0.005720,-0.021311,-0.091343,-0.195557,...,124665.0,43517.0,6583.0,481245.0,210137.0,123948.0,43785.0,26346.0,10821.0,6086.0
17,-0.430204,-0.012116,-0.028651,-0.043785,-0.070246,-0.430226,-0.011812,-0.028496,-0.042138,-0.071508,...,48732.0,24737.0,3529.0,239944.0,96296.0,49073.0,24528.0,12669.0,6431.0,2694.0
2705,-0.350208,-0.003602,-0.011172,-0.028072,-0.053182,-0.349382,-0.003450,-0.010818,-0.027803,-0.051481,...,105385.0,47824.0,6114.0,500391.0,201969.0,105092.0,47944.0,27070.0,12706.0,5782.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,-0.373524,-0.001420,-0.003783,-0.008125,-0.015144,-0.374219,-0.001343,-0.003713,-0.007309,-0.014646,...,88529.0,46375.0,6626.0,467531.0,189423.0,87259.0,45578.0,23877.0,11113.0,6648.0
2762,-0.605758,-0.003195,-0.015823,-0.065967,-0.177299,-0.605206,-0.002883,-0.014599,-0.063928,-0.176441,...,117697.0,50712.0,6616.0,484265.0,215016.0,117036.0,50822.0,24352.0,9862.0,6796.0
1709,-0.661071,-0.002580,-0.006291,-0.013561,-0.045222,-0.656892,-0.002519,-0.006225,-0.012975,-0.043624,...,111435.0,51941.0,4684.0,537664.0,174399.0,109724.0,51131.0,28775.0,12778.0,5524.0
889,-0.236277,-0.000263,-0.000650,-0.001713,-0.004539,-0.236292,-0.000250,-0.000644,-0.001759,-0.004637,...,91470.0,48798.0,5130.0,406654.0,223626.0,90742.0,47946.0,27494.0,12978.0,7313.0


In [5]:
# split dataset into target (y) and predictors (x)
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]

In [6]:
# get wavelet predictors
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values

In [7]:
wavelet_predictor_labels_wf_outliers= numpy.asarray([
    '25th_percentile_db8_cA7', '5th_percentile_db8_cA7', '5th_percentile_db8_cD7',
    '75th_percentile_db8_cA7', 'mcr_db4_cD1', 'mcr_db4_cD2', 'mcr_db8_cD2',
    'mcr_db8_cD1', 'mcr_db5_cD2','mcr_db5_cD1','zcr_db8_cD1'])
wavelet_predictor_labels_wo_outliers = [i for i in wavelet_predictor_labels \
                                 if i not in wavelet_predictor_labels_wf_outliers]

In [8]:
# Create standardization and normalization pipelines
preprocess_sp = standardization_pipeline(
    wavelet_predictor_labels,
    wavelet_predictor_labels_wf_outliers,
    wavelet_predictor_labels_wo_outliers)
preprocess_np = normalization_pipeline(
    wavelet_predictor_labels)

### Split & Transform Data

In [9]:
# split into sets: training 80% & testing 20% of total dataset
X_train, X_test, y_train, y_test = train_test_split(
    wavelet_predictors, y, test_size=0.2, shuffle=True)

In [10]:
# split into sets: training 80% & testing 20% of training set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=True)

In [11]:
# fit standardization pipeline estimator with training data and tranform test set
pipeline_estimator_sp = preprocess_sp.fit(X_train)
X_val_sp = pipeline_estimator_sp.transform(X_val)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

/Users/macbookretina/Desktop/automatic-music-genre-classification/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/macbookretina/Desktop/automatic-music-genre-classification/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/macbookretina/Desktop/automatic-music-genre-classificat

(668, 252, 1)

In [12]:
# fit normalization pipeline estimator with training data and tranform test set
pipeline_estimator_np = normalization_pipeline(
    wavelet_predictor_labels).fit(X_train)
X_val_np = pipeline_estimator_np.transform(X_val)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(668, 252, 1)

In [ ]:
# X_train_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
#                                   ).fit_transform(X_train)
# X_test_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
#                                   ).fit_transform(X_test)
# X_test_sp.shape

In [ ]:
# X_train_np = normalization_pipeline(wavelet_predictor_labels
#                                   ).fit_transform(X_train)
# X_test_np = normalization_pipeline(wavelet_predictor_labels
#                                   ).fit_transform(X_test)
# X_test_np.shape

In [ ]:
_, ncols = X_train.shape

In [ ]:
# one hot encoding our labels
y_train = to_categorical(y_train, 3)
y_val = to_categorical(y_val, 3)
y_test = to_categorical(y_test, 3)

In [ ]:
y_train.shape, y_val.shape, y_test.shape

## Predictive Modelling

In [ ]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [ ]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__learning_rate': [3, 1, .3, .1, .03, .01, .003, .001, .0003, .0001]
}

### FNN With Standardized Input

In [ ]:
def train_model(name, batch_size=32, epochs=100):
    """
    
    """
    # wrap the function with keras wrapper
    fnn = KerasClassifier(build_fn=set_shape_create_model(
        name, ncols))
    # create pipeline estimator
    pipeline = Pipeline([
        ('preprocess', pipeline_estimator_sp),
        ('clf', fnn)
    ])
    # instantiate RandomizedSearchCV
    # if you're not using a GPU, you can set n_jobs to something other than 1
    rscv = RandomizedSearchCV(pipeline, param_dist, cv=3, n_jobs=1)
    # learn model
    search = rscv.fit(X_train, y_train, clf__batch_size=batch_size,
                    clf__validation_data=(X_val_sp, y_val),
                    clf__epochs=epochs, clf__callbacks=[early_stopping_cb])
    
    print(search.best_score_, search.best_params_)
    return search.best_score_, search.best_params_, search.best_estimator_

In [ ]:
fnn1_best_score, fnn1_best_params, fnn1_best_estimator = train_model('fnn_1')

In [ ]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [ ]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [ ]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [ ]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_1.best_score_

In [ ]:
search_1.best_params_

In [ ]:
search_1.best_estimator_

### FNN With Normalized Input

In [ ]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [ ]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [ ]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [ ]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_2.best_score_

In [ ]:
search_2.best_params_

In [ ]:
search_2.best_estimator_

### CNN With Standardized Input

In [ ]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [ ]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [ ]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [ ]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [ ]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_3.best_score_

In [ ]:
search_3.best_params_

In [ ]:
search_3.best_estimator_

### CNN With Normalized Input

In [ ]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [ ]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [ ]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [ ]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

In [ ]:
search_4.best_score_

In [ ]:
search_4.best_params_

In [ ]:
search_4.best_estimator_